In [1]:
#skip warning
import warnings
warnings.filterwarnings('ignore')
#connect to drive
from google.colab import drive  
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install fasttext #install library fasttext untuk word/token/term representasi
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz #download model fasttext bahasa inggris
!gunzip cc.en.300.bin.gz

     |████████████████████████████████| 71kB 3.5MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3095666 sha256=4162e83061b8eb7a6def72ae83ba8fb3bbfc6aa45906b1b04145092b691eaffb
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext
--2021-05-17 06:59:44--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  23.2MB/s    in 3m 9s   

2021-05-17 07:02:53 (22.8 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [3]:
import fasttext
model = fasttext.load_model("cc.en.300.bin") #load model

In [4]:
import numpy as np
import pandas as pd
import math
import re
from scipy.spatial import distance #library untuk cosine similarity

In [5]:
#baca kumpulan kata cultures
def create_dictionary(cultures) :
  path = '/content/drive/MyDrive/Skripsi_Putri/Data_kata.csv'
  list_word = pd.read_csv(path,header=0)
  dict_vect = dict()
  #membersikan kata
  for culture in cultures :
    list_word[culture] = list_word[culture].str.replace('_',' ') #ubah _ dengan spasi
    list_word[culture] = list_word[culture].str.replace('-','') #hapus -
    list_word[culture] = list_word[culture].str.lower() #lowercase
    cul_dict = dict()
    for word in list_word[culture] :
      if type(word) is not str and math.isnan(word) : continue
      cul_dict[word] = model.get_sentence_vector(word) # word to vector
    dict_vect[culture] = cul_dict #save
  return dict_vect
cultures = ['Innovation','Integrity','Quality','Respect','Teamwork']

#mengukur tingkat kemiripan antar token dengan kata pada kumpulan cultures
def similarity(dict_vect,dataset,key) :
  tokens = dataset.copy()
  print("progress %s_%s.pdf"%(key,tahun))
  res_per_docs = dict()
  for token in tokens :
    sent2vect = model.get_sentence_vector(token) #ubah token jadi vector
    max_similar = -1000
    similar_culture = ""
    word_culture = ""
    for culture in dict_vect :
      for word in dict_vect[culture] :
        if (len(token.split(' ')) != len(word.split())) : #tidak perlu mengecek yang banyak word nya berbeda(akselerasi program)
          continue
        dist = 1 - distance.cosine(sent2vect, dict_vect[culture][word]) #menghitung cosine similarity
        #mencari yang similaritynya tertinggi
        if (dist > max_similar) :
          max_similar = dist
          similar_culture = culture
          word_culture = word
        # print(word,culture,token)
    res_per_docs[token] = (similar_culture,word_culture,max_similar) #save hasil
  return res_per_docs

In [6]:
dict_vect = create_dictionary(cultures) #baca kumpulan kata cultures

In [7]:
#Baca Token untuk setiap sektor
import json
#set sektor dan tahun
sektor = 'IDXTRANS'
tahun = '2018'
path = '/content/drive/MyDrive/Skripsi_Putri/Hasil/Tokens/%s_%s.json'%(sektor,tahun)
with open(path, 'r') as fp:
  pdf_tokens = json.load(fp)

for key in pdf_tokens :
 pdf_tokens[key] = list(set(pdf_tokens[key])) #filter token menjadi unik pertokennya.

In [8]:
print(pdf_tokens.keys())

dict_keys(['AKSI', 'APOL', 'ASSA', 'BIRD', 'BLTA', 'BPTR', 'CMPP', 'DEAL', 'GIAA', 'HELI', 'IATA', 'MIRA', 'NELY', 'SAFE', 'SAPX', 'SDMU', 'SMDR', 'TAXI', 'TMAS', 'TNCA', 'TRUK', 'WEHA'])


In [11]:
keys = list(pdf_tokens.keys())
idx = keys.index('AKSI') #atur mulai dari yang belum dikerjakan
import json
for key in keys[idx:] :
  dict_result = dict()
  dict_result = similarity(dict_vect,pdf_tokens[key],key) #mengukur similarity
  with open('/content/drive/MyDrive/Skripsi_Putri/Hasil/Similarity/'+sektor+'/'+key+'_'+tahun+'.json', 'w') as fp:
    json.dump(dict_result, fp) #save hasil tiap perusahaan

progress AKSI.pdf
progress APOL.pdf
progress ASSA.pdf
progress BIRD.pdf
progress BLTA.pdf
progress BPTR.pdf
progress CMPP.pdf
progress DEAL.pdf
progress GIAA.pdf
progress HELI.pdf
progress IATA.pdf
progress MIRA.pdf
progress NELY.pdf
progress SAFE.pdf
progress SAPX.pdf
progress SDMU.pdf
progress SMDR.pdf
progress TAXI.pdf
progress TMAS.pdf
progress TNCA.pdf
progress TRUK.pdf
progress WEHA.pdf
